In [9]:
%pip install dotenv
%pip install langchain_openai
%pip install langchain
%pip install langgraph
%pip install ddgs
%pip install utils
%pip install langchain-tavily
%pip install langchain-core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
if os.path.exists(".env"):
    load_dotenv()
    print("Environment variables loaded from .env file.")


# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API Key loaded successfully!")
else:
    print("API Key not found. Please check your .env file.")

Environment variables loaded from .env file.
API Key loaded successfully!


In [8]:
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END, MessagesState
from IPython.display import display, Image
from langchain_core.tools import tool
from ddgs import DDGS
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_tavily import TavilySearch
import json

In [4]:
# Intialize chat model via LangChain
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [15]:
cv_text = str(input())

In [16]:
job_description = str(input())

In [17]:
cv_title = str(input())

# CV Optimizer Agent

# Message Composer Agent

## Tools for Message Composer Agent

In [12]:
@tool
def research_recruiter(recruiter_name: str, company_name: str) -> str:
    """
    Research a specific recruiter at a company, and the company.
    Args:
        recruiter_name: Full name of the recruiter
        company_name: Company they work at
    Returns:
        Background info, recent posts, interests, company's recent news
    """
    query = f"{recruiter_name} {company_name} recruiter LinkedIn"
    response = TavilySearch(max_results=5, api_key="TAVILY_API_KEY").invoke(query)

    return response

@tool
def analyze_role_alignment(cv_text: str, job_description: str) -> str:
    """
    Analyze alignment between CV and job description.
    Returns key strengths and skills to highlight in the message.
    """
    analysis_prompt = f"""
    Analyze the following CV against the job description.
    Return a JSON object with:
    - "key_strengths": [list of 3 to 4 matching strengths]
    - "keywords_to_mention": [3 to 4 important keywords from job description found in CV]
    
    CV:
    {cv_text}
    
    Job Description:
    {job_description}
    """
    
    response = chat.invoke(analysis_prompt)
    return response.content

def craft_personalized_message(
    recruiter_name: str,
    company_name: str,
    company_info: str,
    recruiter_info: str,
    alignment_analysis: str,
    cv_title: str
) -> str:
    """
    Craft a personalized, engaging message incorporating company news,
    recruiter insights, and CV alignment.
    """
    crafting_prompt = f"""
    Write a professional but personable message to a recruiter. 
    
    Guidelines:
    - Start with a personalized greeting mentioning something specific about them,
    - Reference recent company news or initiatives (show you've done research),
    - Highlight 1-2 specific CV strengths that match the role,
    - Keep it concise (3-4 sentences max),
    - Include a call-to-action about reviewing the attached CV,
    - Tone: Professional but warm, and not desperate.
    
    Context:
    Recruiter: {recruiter_name}
    Company: {company_name}
    Role: {cv_title}
    
    Company Recent News/Info:
    {company_info}
    
    Recruiter Background:
    {recruiter_info}
    
    CV-Role Alignment:
    {alignment_analysis}
    
    Generate 2 different versions with slightly different tones.
    """
    
    response = chat.invoke(crafting_prompt)
    return response.content

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Actual message writer agent:

tools = [
    research_recruiter,
    analyze_role_alignment,
    craft_personalized_message
]

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert recruiter communication specialist. 
    Your goal is to help job seekers write highly personalized messages to recruiters.
    
    Follow this workflow:
    1. Research the company and find recent news/initiatives
    2. Research the specific recruiter
    3. Analyze how the CV aligns with the job description
    4. Craft personalized message(s) incorporating all insights
    
    Always prioritize personalization and genuine interest over generic templates."""),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


input_variables=['agent_scratchpad', 'input'] input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_cor

In [24]:
result = craft_personalized_message(
    recruiter_name="Sarah Chen",
    company_name="Anthropic",
    company_info="",
    recruiter_info= "",
    alignment_analysis= "",
    cv_title= "Machine Learning Expert"
)

print(result)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3gEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}}

# Interview Questions Agent